In [1]:
import matplotlib.pyplot as plt_func
from sklearn.model_selection import LeaveOneOut as LOO
from sklearn.linear_model import LinearRegression as LinReg
import statsmodels.api as sm_func

In [2]:
def removeColumnsFromList(df, columnsToRemove):
    """
    Return a list of columns names excluding the names in the list 
    `columnsToKeep`.
    
    Args:
        df: pandas.core.frame.DataFrame
            The DataFrame used to produce the list of column names. 
        
        columnsToRemove: iterable
            An iterable object that has the names as elements that
            will be excluded from the returned list.
    Returns:
        list: The aforementioned column names.
    """
    columns = df.columns.tolist()
    for column in columnsToRemove:
        try:
            columns.remove(column)
        except ValueError as err:
            if not 'list.remove(x): x not in list' in str(err):
                raise

    return columns

<p>We could easily reproduce these type of plots <a href="https://www.statsmodels.org/dev/examples/notebooks/generated/regression_plots.html">very closely</a>, with <a href="http://www.statsmodels.org/dev/examples/notebooks/generated/regression_diagnostics.html">additional regression diagnostics</a>, using the <code>statsmodels</code> library, however I would like to have more control, so I decided to plot most of them manually using <code>matplotlib</code>.

In [3]:
def createResidualPlots(df_X, df_Y, fitted_model, list_of_indices=[], width=7, height=3):
    """
    This function returns various residual plots for the fitted model.
    
    For linear regressions, the first two plots are plots of the 
    residuals and the square root of the absolute standardized residuals
    vs the predictor. For the multiple regression fit, we instead plots 
    the residuals and the square root of the absolute standardized 
    residuals vs the fitted values. The third plot is a QQ plot of the
    quantiles of the standardized residuals vs the quantiles of the 
    normal distribution, and a 45-degree reference line is also plotted 
    for comparison (see also 
    https://seankross.com/2016/02/29/A-Q-Q-Plot-Dissection-Kit.html). The 
    final plot is a leverage plot of the standardized residuals.
    
    Args:
        df_X: pandas.core.frame.DataFrame
            The DataFrame should hold the data of independent variables
            (including a column for the 'Intercept' set equal to one).
            Each row in the DataFrame represents an individual sample 
            point, with the successive columns corresponding to the 
            independent variables and their specific values for that 
            sample point. 
        df_Y: pandas.core.frame.DataFrame or pandas.core.series.Series
            This should be a pandas Series of DataFrame of one column,
            which holds the data of the dependent variable.        
        fitted_model: statsmodels.regression.linear_model.RegressionResultsWrapper
            This statsmodels class summarizes the fit of a linear regression model
            that has been fitted with df_X and df_Y.
        list_of_indices: list, default list()
            A list that hold indices indicating which data point(s) want to 
            be colored differently to distinguish those point(s) from the 
            rest of the data.
        width: float, default 7
            The width of each subplot.
        height: float, default 3
            The height of each subplot.
    """ 
    descriptiveColumns = df_X.columns.tolist()
    try:
        descriptiveColumns.remove('Intercept')
    except ValueError as err:
        if not 'list.remove(x): x not in list' in str(err):
            raise
            
    assert len(descriptiveColumns) >= 1, f'descriptiveColumns = {descriptiveColumns}'
    approach = 'simple' if len(descriptiveColumns) == 1 else 'multivariable'
    
    sr_Y_hat = fitted_model.predict(df_X)
    residual = np.squeeze(df_Y.to_numpy()) - sr_Y_hat
    from sklearn import preprocessing
    standardized_residual = preprocessing.scale(residual)
    
    X = df_X.to_numpy()
    try:
        H = X @ np.linalg.inv(X.transpose() @ X) @ X.transpose()
        leverage = H.diagonal() 
    except np.linalg.LinAlgError as err:
        if not 'Singular matrix' in str(err):
            raise
            
        leverage = None
    
    numberOfSubplots = 3 if leverage is None else 4
    fig, axes = plt_func.subplots(numberOfSubplots, 1, constrained_layout=True, figsize=(width, height*numberOfSubplots))
    if approach == 'simple':
        descriptive = descriptiveColumns[0]
        X_plot = df_X[descriptive].to_numpy()
    else:
        descriptive = 'fitted values'
        X_plot = sr_Y_hat.to_numpy()
        
    mask_special_indices = np.zeros(residual.shape[0], dtype=bool)
    mask_special_indices[list_of_indices] = True
    
    from matplotlib import colors
    default_colors = plt_func.rcParams['axes.prop_cycle'].by_key()['color']
    cmap = colors.ListedColormap(default_colors[:2])
    
    _ = axes[0].scatter(x=X_plot, y=residual, c=mask_special_indices, cmap=cmap)
    _ = axes[0].set_xlabel(descriptive)
    _ = axes[0].set_ylabel('residuals')
    _ = axes[0].set_title(f'residual plot for the linear regression')
    
    _ = axes[1].scatter(x=X_plot, y=np.absolute(standardized_residual)**0.5, c=mask_special_indices, cmap=cmap)
    _ = axes[1].set_xlabel(descriptive)
    _ = axes[1].set_ylabel(r'$\sqrt{\left|\mathrm{standardized \,\,\, residuals}\right|}$')
    _ = axes[1].set_title(r'$\sqrt{\left|\mathrm{standardized \,\,\, residuals}\right|}$ for the linear regression')
        
    n = sr_Y_hat.shape[0] + 1
    q_list = np.linspace(start=1/n, stop=1, num=n)
    quantiles_data = np.sort(standardized_residual)
    from scipy import stats
    quantiles_theoretical = stats.norm.ppf(q_list)[:-1]  # remove infinity from array
    _ = axes[2].scatter(x=quantiles_theoretical, y=quantiles_data, c=mask_special_indices, cmap=cmap)
    x_min, x_max = axes[2].get_xlim()
    y_min, y_max = axes[2].get_ylim()
    axes[2].plot((x_min, x_max), (y_min, y_max), color='black', label='45-degree line')
    _ = axes[2].set_xlabel('normal distribution quantiles')
    _ = axes[2].set_ylabel('standardized residuals quantiles')
    _ = axes[2].set_title('normal qq plot')
    _ = axes[2].legend()
    
    if not leverage is None:
        _ = axes[3].scatter(x=leverage, y=standardized_residual, c=mask_special_indices, cmap=cmap)
        _ = axes[3].set_xlabel('leverage')
        _ = axes[3].set_ylabel('standardized residuals')
        _ = axes[3].set_title(f'standardized residuals vs leverage')

In [4]:
def createSimpleLinearRegressionPlot(df_X, df_Y, fitted_model, alpha=0.05, width=8, height=3):
    """
    This function returns a scatter plot of the response and the predictor. 
    Furthermore, the simple linear regression line is shown with an 
    associated confidence and prediction interval of 1-alpha.
    
    Args:
        df_X: pandas.core.frame.DataFrame
            The DataFrame should hold the data of the independent variable.
            Each row in the DataFrame represents an individual sample point.
        df_Y: pandas.core.frame.DataFrame
            This should be a DataFrame of one column,
            which holds the data of the dependent variable.            
        fitted_model: statsmodels.regression.linear_model.RegressionResultsWrapper
            This statsmodels class summarizes the fit of a linear regression model
            that has been fitted with df_X and df_Y.
        alpha: float, default 0.05
            This prediction and confidence intervals that are being shown are
            of 1-alpha (e.g., alpha=0.05 corresponds to 95% confidence).
        width: float, default 8
            The width of the plot.
        height: float, default 3
            The height of the plot.        
    """ 
    
    assert df_Y.shape[1] == 1
    dependent = df_Y.columns[0]
    descriptiveColumns = df_X.columns.tolist()
    try:
        descriptiveColumns.remove('Intercept')
        contains_intercept = True
    except ValueError as err:
        if not 'list.remove(x): x not in list' in str(err):
            raise

        contains_intercept = False
        
    independent = descriptiveColumns[0]
    independent_arr = np.linspace(start=df_X[independent].min(), stop=df_X[independent].max(), num=df_X.shape[0])
    if contains_intercept:
        df_X_pred = pd.DataFrame({
            'Intercept': np.ones(shape=(df_X.shape[0], ), dtype=int), 
            independent: independent_arr,
        }, columns = ['Intercept', independent])
    else:
        df_X_pred = pd.DataFrame({independent: independent_arr})
        
    sr_Y_pred = fitted_model.predict(df_X_pred)
    
    # get prediction intervals
    try:
        from statsmodels.sandbox.regression.predstd import wls_prediction_std
        std_err_prediction, lower_pred_int, upper_pred_int = wls_prediction_std(fitted_model, exog=df_X_pred, alpha=alpha)
    except AttributeError as err:
        if not '\'float\' object has no attribute \'sqrt\'' in str(err):
            raise

        std_err_prediction, lower_pred_int, upper_pred_int = None, None, None

    # get confidence intervals
    try:
        result = fitted_model.get_prediction(df_X_pred)
        conf_int = result.conf_int(alpha=alpha)
        lower_conf_int, upper_conf_int = conf_int[:, 0], conf_int[:, 1]
    except AttributeError as err:
        if not '\'float\' object has no attribute \'sqrt\'' in str(err):
            raise   

        lower_conf_int, upper_conf_int = None, None
        
    fig, ax = plt_func.subplots(constrained_layout=True, figsize=(width, height))
    _ = ax.scatter(df_X[independent], df_Y, label='training data')
    _ = ax.plot(df_X_pred[independent], sr_Y_pred, '-', color='darkorchid', linewidth=2, label='prediction')
    if not lower_conf_int is None and not upper_conf_int is None:
        _ = ax.fill_between(df_X_pred[independent], lower_conf_int, upper_conf_int, color='#888888', alpha=0.4, label=f"confidence interval ({int((1-alpha)*100)}%)")
    if not lower_pred_int is None and not upper_pred_int is None:
        _ = ax.fill_between(df_X_pred[independent], lower_pred_int, upper_pred_int, color='#888888', alpha=0.1, label=f"prediction interval ({int((1-alpha)*100)}%)")

    _ = ax.legend()
    _ = ax.set_xlabel(independent)
    _ = ax.set_ylabel(dependent)
    _ = ax.set_title(f'regression of prediction vs training data')
    _ = ax.grid(True)

In [5]:
def createSimpleLinearRegressionPlotWithTransformation(df_X, df_Y, fitted_model, df_independent, alpha=0.05, width=8, height=3):
    """
    This function returns a scatter plot of the response and the transformed
    predictor. Furthermore, the simple linear regression line is shown with 
    an associated confidence and prediction interval of 1-alpha.
    
    Args:
        df_X: pandas.core.frame.DataFrame
            The DataFrame should hold the data of the independent variable.
            Each row in the DataFrame represents an individual sample point.
        df_Y: pandas.core.frame.DataFrame
            This should be a DataFrame of one column,
            which holds the data of the dependent variable.
        df_independent: pd.core.frame.DataFrame
            This DataFrame should hold the data of the independent variable before 
            the transformation has been applied to the variable, and will be 
            used to plot the regression instead of the transformed variable in
            the DataFrame df_X.
        fitted_model: statsmodels.regression.linear_model.RegressionResultsWrapper
            This statsmodels class summarizes the fit of a linear regression model
            that has been fitted with df_X and df_Y.
        alpha: float, default 0.05
            This prediction and confidence intervals that are being shown are
            of 1-alpha (e.g., alpha=0.05 corresponds to 95% confidence).
        width: float, default 8
            The width of the plot.
        height: float, default 3
            The height of the plot.        
    """ 
    
    assert df_Y.shape[1] == 1
    dependent = df_Y.columns[0]
    descriptiveColumns = df_X.columns.tolist()
    try:
        descriptiveColumns.remove('Intercept')
        contains_intercept = True
    except ValueError as err:
        if not 'list.remove(x): x not in list' in str(err):
            raise

        contains_intercept = False  
    
    independent = descriptiveColumns[0]
    independent_arr = np.linspace(start=df_X[independent].min(), stop=df_X[independent].max(), num=df_X.shape[0])
    if contains_intercept:
        df_X_pred = pd.DataFrame({
            'Intercept': np.ones(shape=(df_X.shape[0], ), dtype=int), 
            independent: independent_arr,
        }, columns = ['Intercept', independent])
    else:
        df_X_pred = pd.DataFrame({independent: independent_arr})
    
    sr_Y_pred = fitted_model.predict(df_X_pred)

    # get prediction intervals
    try:
        from statsmodels.sandbox.regression.predstd import wls_prediction_std
        std_err_prediction, lower_pred_int, upper_pred_int = wls_prediction_std(fitted_model, exog=df_X_pred, alpha=alpha)
    except AttributeError as err:
        if not '\'float\' object has no attribute \'sqrt\'' in str(err):
            raise

        std_err_prediction, lower_pred_int, upper_pred_int = None, None, None

    # get confidence intervals
    try:
        result = fitted_model.get_prediction(df_X_pred)
        conf_int = result.conf_int(alpha=alpha)
        lower_conf_int, upper_conf_int = conf_int[:, 0], conf_int[:, 1]
    except AttributeError as err:
        if not '\'float\' object has no attribute \'sqrt\'' in str(err):
            raise   

        lower_conf_int, upper_conf_int = None, None

    independent = df_independent.columns[0]
#     df_X = pd.concat([df_X, df_independent], axis=1)
    df_X_pred[independent] = np.linspace(start=df_independent.min(), stop=df_independent.max(), num=df_independent.shape[0])
    
    fig, ax = plt_func.subplots(constrained_layout=True, figsize=(width, height))
    _ = ax.scatter(df_independent, df_Y, label='pre-transformed training data')
    _ = ax.plot(df_X_pred[independent], sr_Y_pred, '-', color='darkorchid', linewidth=2, label='prediction')
    if not lower_conf_int is None and not upper_conf_int is None:
        _ = ax.fill_between(df_X_pred[independent], lower_conf_int, upper_conf_int, color='#888888', alpha=0.4, label=f"confidence interval ({int((1-alpha)*100)}%)")
    if not lower_pred_int is None and not upper_pred_int is None:
        _ = ax.fill_between(df_X_pred[independent], lower_pred_int, upper_pred_int, color='#888888', alpha=0.1, label=f"prediction interval ({int((1-alpha)*100)}%)")

    _ = ax.legend()
    _ = ax.set_xlabel(independent)
    _ = ax.set_ylabel(dependent)
    _ = ax.set_title('regression of prediction vs pre-transformed training data')
    _ = ax.grid(True)

In [6]:
def createPolynomialLinearRegressionPlot(df_X, df_Y, fitted_model, polynomialMap, df_independent=None, alpha=0.05, width=8, height=3):
    """
    This function returns a scatter plot of the response and the predictor. 
    Furthermore, the polynomial regression line is shown with an 
    associated confidence and prediction interval of 1-alpha.
    
    Args:
        df_X: pandas.core.frame.DataFrame
            The DataFrame should hold the data of the exponentiated variable
            used for the polynomial regression. Each row in the DataFrame 
            represents an individual sample point.
        df_Y: pandas.core.frame.DataFrame
            This should be a DataFrame of one column,
            which holds the data of the dependent variable.
        fitted_model: statsmodels.regression.linear_model.RegressionResultsWrapper
            This statsmodels class summarizes the fit of a linear regression model
            that has been fitted with df_X and df_Y.
        polynomialMap: dict
            This argument is used when plotting a polynomial regression. It is a 
            dictionary that must contain the column names of the DataFrame X 
            (excluding the intercept if the model is fitted with an intercept) as
            keys with their associated polynomial degrees as values. For instance,
            let us imagine that the model being fitted takes the form
                y = a + b*x^2 + c*x^5
            then the polynomialMap argument should be
                mapping_powers = {
                    'b': 2,
                    'c': 5,
                }
        df_independent: pd.core.frame.DataFrame, default None
            This argument is only used when plotting a polynomial regression without
            a term of degree 1. The DataFrame should hold the data of the independent 
            variable exponentiated to the power 1, and will be used to plot the 
            regression instead of the transformed variable in the DataFrame df_X.
            A string describing the independent variable of the regression. 
        alpha: float, default 0.05
            This prediction and confidence intervals that are being shown are
            of 1-alpha (e.g., alpha=0.05 corresponds to 95% confidence).
        width: float, default 8
            The width of the plot.
        height: float, default 3
            The height of the plot.        
    """ 
    
    assert df_Y.shape[1] == 1
    dependent = df_Y.columns[0]
    descriptiveColumns = df_X.columns.tolist()
    try:
        descriptiveColumns.remove('Intercept')
        contains_intercept = True
    except ValueError as err:
        if not 'list.remove(x): x not in list' in str(err):
            raise

        contains_intercept = False
        
    assert len(descriptiveColumns) == len(polynomialMap), f'descriptiveColumns = {descriptiveColumns} and polynomialMap = {polynomialMap}'
    sortedMap = [(key, polynomialMap[key]) for key in descriptiveColumns if key in polynomialMap]
    _, polynomialTuple = zip(*sortedMap)
    first_degree_polynomial_term = True if 1 in polynomialTuple else False
        
    if contains_intercept:
        df_X_pred = pd.DataFrame({'Intercept': np.ones(df_X.shape[0], dtype=int)})
    else:
        df_X_pred = pd.DataFrame(index=range(0, df_X.shape[0]))

    if first_degree_polynomial_term:
        index_independent = polynomialTuple.index(1)
        independent = descriptiveColumns[index_independent]
        independent_arr = np.linspace(start=df_X[independent].min(), stop=df_X[independent].max(), num=df_X.shape[0])
        for index, power in enumerate(polynomialTuple):
            column = descriptiveColumns[index]
            if index == index_independent: 
                df_X_pred[independent] = independent_arr
            else:
                df_X_pred[column] = independent_arr**power
    else:
        assert not df_independent is None
        independent = df_independent.columns[0]
        sr_X_independent = df_independent[independent]
        assert isinstance(sr_X_independent, pd.core.series.Series), f'type(sr_X_independent) = {type(sr_X_independent)}'
        X_pred_independent = np.linspace(start=sr_X_independent.min(), stop=sr_X_independent.max(), num=df_X.shape[0])
        for index, power in enumerate(polynomialTuple):
            column = descriptiveColumns[index]
            df_X_pred[column] = X_pred_independent**power

    sr_Y_pred = fitted_model.predict(df_X_pred)

    # get prediction intervals
    try:
        from statsmodels.sandbox.regression.predstd import wls_prediction_std
        std_err_prediction, lower_pred_int, upper_pred_int = wls_prediction_std(fitted_model, exog=df_X_pred, alpha=alpha)
    except AttributeError as err:
        if not '\'float\' object has no attribute \'sqrt\'' in str(err):
            raise

        std_err_prediction, lower_pred_int, upper_pred_int = None, None, None

    # get confidence intervals
    try:
        result = fitted_model.get_prediction(df_X_pred)
        conf_int = result.conf_int(alpha=alpha)
        lower_conf_int, upper_conf_int = conf_int[:, 0], conf_int[:, 1]
    except AttributeError as err:
        if not '\'float\' object has no attribute \'sqrt\'' in str(err):
            raise   

        lower_conf_int, upper_conf_int = None, None

    if not first_degree_polynomial_term:
        df_X = df_X.copy()
        df_X[independent] = sr_X_independent
        df_X_pred[independent] = X_pred_independent
        
    fig, ax = plt_func.subplots(constrained_layout=True, figsize=(width, height))
    _ = ax.scatter(df_X[independent], df_Y, label='training data')
    _ = ax.plot(df_X_pred[independent], sr_Y_pred, '-', color='darkorchid', linewidth=2, label='prediction')
    if not lower_conf_int is None and not upper_conf_int is None:
        _ = ax.fill_between(df_X_pred[independent], lower_conf_int, upper_conf_int, color='#888888', alpha=0.4, label=f"confidence interval ({int((1-alpha)*100)}%)")
    if not lower_pred_int is None and not upper_pred_int is None:
        _ = ax.fill_between(df_X_pred[independent], lower_pred_int, upper_pred_int, color='#888888', alpha=0.1, label=f"prediction interval ({int((1-alpha)*100)}%)")

    _ = ax.legend()
    _ = ax.set_xlabel(independent)
    _ = ax.set_ylabel(dependent)
    _ = ax.set_title('regression of prediction vs training data')
    _ = ax.grid(True)

In [7]:
def createConfusionMatrixFromLogisticModel(fitted_model, threshold=0.5, binaryMap={0: 0, 1: 1}):
    """
    This function returns two confusion matrices in terms of absolute 
    numbers and percentages, respectively, based on in-sample data fitted
    with a logistic regression model.
    
    Args:
        fitted_model: statsmodels.discrete.discrete_model.LogitResults
            This statsmodels class summarizes the fit of a logistic 
            regression model.
        threshold: float, default 0.5
            Number between 0 and 1. Threshold above which a prediction
            is considered 1 and below which a prediction is considered 0.
        binaryMap: dictionary, default {0: 0, 1: 1}
            A mapping of the binary 0 and 1 quantative variables to their
            associated qualitative name.
    Returns:
        tuple of two pandas.core.frame.DataFrames: The aforementioned confusion
            matrices, in absolute values and percentages respectively.
    """
   
    # pred_table[i,j] refers to the number of times “i” was observed
    # and the model predicted “j”. Correct predictions are along the diagonal.
    confusion = fitted_model.pred_table(threshold=0.5).astype(int)
    
    index = pd.MultiIndex.from_tuples([('Observed', binaryMap[0]), ('Observed', binaryMap[1])])
    columns = pd.MultiIndex.from_tuples([('Predicted', binaryMap[0]), ('Predicted', binaryMap[1])])
    df_confusion = pd.DataFrame(confusion, columns=columns, index=index)
    
    # TN, FP, FN and TP denote the 'true negative', 'false positive',
    # 'false negative' and 'true positive', respectively.
    TN, FP, FN, TP = confusion[0, 0], confusion[0, 1], confusion[1, 0], confusion[1, 1]
    TNR = TN / (TN + FP)  # true negative rate
    FPR = FP / (TN + FP)  # false positive rate
    FNR = FN / (TP + FN)  # false negative rate
    TPR = TP / (TP + FN)  # true positive rate
    confusion_pct = 100 * np.array([
        [TNR, FPR],
        [FNR, TPR]
    ])
    index_pct = pd.MultiIndex.from_tuples([('Observed (%)', binaryMap[0]), ('Observed (%)', binaryMap[1])])
    columns_pct = pd.MultiIndex.from_tuples([('Predicted (%)', binaryMap[0]), ('Predicted (%)', binaryMap[1])])
    df_confusion_pct = pd.DataFrame(confusion_pct, columns=columns_pct, index=index_pct)
    
    return df_confusion, df_confusion_pct

In [8]:
def createConfusionMatrixFromOutOfSampleData(df, binaryMap={0: 0, 1: 1}):
    """
    This function returns two confusion matrices in terms of absolute 
    numbers and percentages, respectively, based on out-of-sample data.
    
    Args:
        df: pandas.core.frame.DataFrame
            The DataFrame used to produce the confusion matrices. It 
            should have a column named 'Observed' and one column named
            'Predicted' which contains the binary values (0 or 1) of 
            the observed and predicted data, respectively.
        binaryMap: dictionary, default {0: 0, 1: 1}
            A mapping of the binary 0 and 1 quantative variables to their
            associated qualitative name.
    Returns:
        tuple of two pandas.core.frame.DataFrames: The aforementioned confusion
            matrices, in absolute values and percentages respectively.
    """
    
    TP = np.sum(np.where((df['Observed'] == 1) & (df['Predicted'] == 1), 1, 0))
    TN = np.sum(np.where((df['Observed'] == 0) & (df['Predicted'] == 0), 1, 0))
    FP = np.sum(np.where((df['Observed'] == 0) & (df['Predicted'] == 1), 1, 0))
    FN = np.sum(np.where((df['Observed'] == 1) & (df['Predicted'] == 0), 1, 0))
    
    confusion = np.array([
        [TN, FP],
        [FN, TP]
    ])
    
    index = pd.MultiIndex.from_tuples([('Observed', binaryMap[0]), ('Observed', binaryMap[1])])
    columns = pd.MultiIndex.from_tuples([('Predicted', binaryMap[0]), ('Predicted', binaryMap[1])])
    df_confusion = pd.DataFrame(confusion, columns=columns, index=index)
    
    TNR = TN / (TN + FP)  # true negative rate
    FPR = FP / (TN + FP)  # false positive rate
    FNR = FN / (TP + FN)  # false negative rate
    TPR = TP / (TP + FN)  # true positive rate
    confusion_pct = 100 * np.array([
        [TNR, FPR],
        [FNR, TPR]
    ])
    index_pct = pd.MultiIndex.from_tuples([('Observed (%)', binaryMap[0]), ('Observed (%)', binaryMap[1])])
    columns_pct = pd.MultiIndex.from_tuples([('Predicted (%)', binaryMap[0]), ('Predicted (%)', binaryMap[1])])
    df_confusion_pct = pd.DataFrame(confusion_pct, columns=columns_pct, index=index_pct)
    
    return df_confusion, df_confusion_pct

In [9]:
def stepFunctionChooseOptimalCuts(df_X, df_Y, total_cuts):
    """
    This function fits a step function to a data set, and 
    performs cross-validation to choose the optimal number
    of cuts. 
    
    Args:
        df_X: pandas.core.frame.DataFrame
            This should be a DataFrame of one column,
            which holds the data of the independent 
            variable used to fit a step function. Each 
            row in the DataFrame represents an individual
            sample point.
        df_Y: pandas.core.frame.DataFrame
            This should be a DataFrame of one column,
            which holds the data of the dependent variable.
        total_cuts: int
            This number represents the total numbers of cuts
            this function will use to obtain the test MSE.
    """
    assert df_X.shape[1] == 1, f'df_X.shape = {df_X.shape}'
    loocv = LOO() # LeaveOneOut
    results = {}

    total_bins = total_cuts + 1
    independent = df_X.columns[0]
    n = df_X.shape[0]
    for no_bins in range(2, total_bins + 1):
        MSE = 0
        counter = 0
        for train_index, test_index in loocv.split(df_X):
            df_X_train, df_X_test = df_X.iloc[train_index], df_X.iloc[test_index]
            df_Y_train, df_Y_test = df_Y.iloc[train_index], df_Y.iloc[test_index]

            df_cut_train, bins_train = pd.cut(df_X_train[independent], bins=no_bins, retbins=True, right=False)
            df_steps_train = pd.concat([df_X_train, df_cut_train, df_Y_train], axis=1)
            df_steps_train.columns = ['independent', 'independent_cuts', 'dependent']

            # Create dummy variables for the age groups
            df_steps_dummies_train = pd.get_dummies(df_steps_train['independent_cuts'])
            # delete first column (see footnote on page 269 of 'An Introduction to Statistical Learning')
            df_steps_dummies_train = df_steps_dummies_train.drop(df_steps_dummies_train.columns[0], axis=1)  
            df_steps_dummies_train = sm_func.add_constant(df_steps_dummies_train)

            fitted = sm_func.GLM(df_Y_train, df_steps_dummies_train).fit()

            # Put the test data in the same bins as the training data.
            bin_mapping_test = str(np.digitize(df_X_test, bins_train)[0, 0])
            # start range from 2 because we ignore the first column (see footnote on page 269 of 'An Introduction to Statistical Learning')
            df_X_test2 = pd.DataFrame({str(i): [0] for i in range(2, no_bins + 1)})
            df_X_test2.insert(0, 'Intercept', 1)
            if bin_mapping_test != '1' and bin_mapping_test != '0': # bin_mapping_test == '0' happens when the test value is smaller than any of the bins, so gets put in the zeroth bin
                # if bin_mapping_test == '1', then the value is put in the first bin, that we ignore (see footnote on page 269 of 'An Introduction to Statistical Learning')
                # because when bin_mapping_test == '1', equation (7.5) on page 269 reduces to:
                # y_i = \beta_0
                # so this case is already absorbed in the intercept, and we can safely ignore it
                try:
                    assert bin_mapping_test in df_X_test2.columns, f'bin_mapping_test = {bin_mapping_test}, df_X_test.iloc[0, 0] = {df_X_test.iloc[0, 0]} and df_X_test2.columns = {df_X_test2.columns}'
                except AssertionError:
                    if bin_mapping_test != str(int(no_bins + 1)): # bin_mapping_test == str(no_bins + 1) happens when the test value is larger than any of the bins, so gets put in the (no_bins + 1) bin
                        raise
                        
                    bin_mapping_test = str(no_bins)
                    
                df_X_test2[bin_mapping_test] = 1

            sr_Y_pred = fitted.predict(df_X_test2)
            MSE += (df_Y_test.iloc[0, 0] - sr_Y_pred.iloc[0])**2 

        cuts = no_bins - 1
        results[cuts] = MSE / n
        
    cuts_lst = list(results.keys())
    MSE_lst = list(results.values())
    std_dev = np.std(MSE_lst)
    min_mse = min(results, key=results.get)
    min_val = results[min_mse]
    cuts_lst.remove(min_mse)  # we plot these 'best results' with a different marker
    MSE_lst.remove(min_val)  # we plot these 'best results' with a different marker

    numberOfSubplots = 1
    fig, ax1 = plt_func.subplots(1, numberOfSubplots, constrained_layout=True, figsize=(8*numberOfSubplots, 4))
    _ = ax1.scatter(cuts_lst, MSE_lst)
    _ = ax1.scatter(min_mse, min_val, marker='x', s=40, c=plt_func.rcParams['axes.prop_cycle'].by_key()['color'][0], label='smallest MSE')
    min_val_2std = min_val - 0.2*std_dev
    _ = ax1.axhline(y=min_val_2std, linestyle='dashed', linewidth=0.5, c=plt_func.rcParams['axes.prop_cycle'].by_key()['color'][1], label='0.2 standard deviation')
    _ = ax1.axhline(y=min_val + 0.2*std_dev, linestyle='dashed', linewidth=0.5, c=plt_func.rcParams['axes.prop_cycle'].by_key()['color'][1])
    _ = ax1.set_xlabel(r'number of cut points')
    _ = ax1.set_ylabel('Cross-validation MSE')
    _ = ax1.legend()
    
    diff = max(MSE_lst) - min_val_2std
    if diff > 70:
        _ = ax1.set_ylim(ymin = min_val_2std - (min_val_2std % 10) - 10)
    elif diff > 40:
        _ = ax1.set_ylim(ymin = min_val_2std - (min_val_2std % 10))
    else:
        _ = ax1.set_ylim(ymin = min_val_2std - (min_val_2std % 1) - 1)

In [10]:
def plotFittedStepFunction(df_X, df_Y, cuts, alpha=0.05):
    """
    This function returns a scatter plot of the response and the predictor. 
    Furthermore, fitted step functions are shown with their associated 
    confidence interval of 1-alpha.
    
    Args:
        df_X: pandas.core.frame.DataFrame
            This should be a pandas DataFrame of one column, that holds
            the data of independent variable. Each row in the DataFrame 
            represents an individual sample point.
        df_Y: pandas.core.frame.DataFrame or pandas.core.series.Series
            This should be a pandas DataFrame of one column,
            which holds the data of the dependent variable. 
        cuts: int
            Integer representing the number of cuts that should be created
            to fit the step functions.
        alpha: float, default 0.05
            This prediction and confidence intervals that are being shown are
            of 1-alpha (e.g., alpha=0.05 corresponds to 95% confidence).
    """ 
    no_bins = cuts + 1
    independent = df_X.columns[0]
    dependent = df_Y.columns[0]

    df_cut, bins = pd.cut(df_X[independent], bins=no_bins, retbins=True, right=False)
    df_steps = pd.concat([df_X, df_cut, df_Y], axis=1)
    df_steps.columns = ['independent', 'independent_cuts', 'dependent']

    # Create dummy variables for the age groups
    df_steps_dummies = pd.get_dummies(df_steps['independent_cuts'])
    # delete first column (see footnote on page 269 of 'An Introduction to Statistical Learning')
    df_steps_dummies = df_steps_dummies.drop(df_steps_dummies.columns[0], axis=1)  
    df_steps_dummies = sm_func.add_constant(df_steps_dummies)

    fitted_model = sm_func.GLM(df_Y, df_steps_dummies).fit()

    X_pred = np.linspace(df_X.min().iloc[0], df_X.max().iloc[0], num=df_X.shape[0], endpoint=True)
    # Put the test data in the same bins as the training data.
    bin_mapping = np.digitize(X_pred.ravel(), bins)
    # Get dummies, drop first dummy category, add constant
    df_X_pred = sm_func.add_constant(pd.get_dummies(bin_mapping).drop(1, axis = 1))
    # Predict the value of the generated ages using the linear model
    sr_Y_pred = fitted_model.predict(df_X_pred)

    numberOfSubplots = 1
    fig, ax1 = plt_func.subplots(1, numberOfSubplots, constrained_layout=True, figsize=(8*numberOfSubplots, 4))
    _ = ax1.scatter(df_X, df_Y, label='training data', s=0.5)
    _ = ax1.plot(X_pred, sr_Y_pred, '-', color='darkorchid', linewidth=2, label='prediction')

    # get confidence intervals
    try:
        result = fitted_model.get_prediction(df_X_pred)
        conf_int = result.conf_int(alpha=alpha)
        lower_conf_int, upper_conf_int = conf_int[:, 0], conf_int[:, 1]
    except AttributeError as err:
        if not '\'float\' object has no attribute \'sqrt\'' in str(err):
            raise

        lower_conf_int, upper_conf_int = None, None

    if not lower_conf_int is None and not upper_conf_int is None:
        _ = ax1.fill_between(X_pred, lower_conf_int, upper_conf_int, color='#888888', alpha=0.4, label=f"confidence interval ({int((1-alpha)*100)}%)")
        
    _ = ax1.set_xlabel(independent)
    _ = ax1.set_ylabel(dependent)
    _ = ax1.set_ylim(ymin = 0)
    _ = ax1.legend()
    _ = ax1.set_title(f'fitted step function of prediction vs training data')

In [11]:
def polynomialRegressionChooseOptimalDegree(df_X, df_Y, total_degrees):
    """
    This function fits a polynomial regression to a data set, 
    and performs cross-validation to choose the optimal number
    of degrees of the polynomial fit. 
    
    Args:
        df_X: pandas.core.frame.DataFrame
            This should be a DataFrame of one column,
            which holds the data of the independent 
            variable used to fit a step function. Each 
            row in the DataFrame represents an individual
            sample point.
        df_Y: pandas.core.frame.DataFrame
            This should be a DataFrame of one column,
            which holds the data of the dependent variable.
        total_degrees: int
            This number represents the total numbers of degrees
            this function will use to obtain the test MSE.
    """
    assert df_X.shape[1] == 1, f"'Intercept' should not be included"
    df_X = df_X.copy()
    independent = df_X.columns[0]
    for i in range(2, total_degrees + 1):
        variable_name = independent + str(i)
        df_X[variable_name] = df_X[independent]**i

    MSE = np.sum((df_Y - df_Y.mean().iloc[0])**2).iloc[0] / df_Y.shape[0]
    best_submodels = {
        0: (['Intercept'], MSE),  # null model
    }

    n, p = df_X.shape
    features_lst = df_X.columns.tolist()
    results = {}
    loocv = LOO() # LeaveOneOut
    for k in range(1, p + 1):
        descriptiveColumns = features_lst[:k]
        MSE = 0
        for train_index, test_index in loocv.split(df_X):
            df_X_train, df_X_test = df_X[descriptiveColumns].iloc[train_index], df_X[descriptiveColumns].iloc[test_index]
            df_Y_train, df_Y_test = df_Y.iloc[train_index], df_Y.iloc[test_index]

            model = LinReg()
            _ = model.fit(df_X_train, df_Y_train)

            Y_pred = model.predict(df_X_test)
            MSE += (df_Y_test.iloc[0, 0] - Y_pred[0, 0])**2        

        results[k] = MSE / n

    degrees_lst = list(results.keys())
    MSE_lst = list(results.values())
    std_dev = np.std(MSE_lst)
    min_mse = min(results, key=results.get)
    min_val = results[min_mse]
    degrees_lst.remove(min_mse)  # we plot these 'best results' with a different marker
    MSE_lst.remove(min_val)  # we plot these 'best results' with a different marker

    numberOfSubplots = 1
    fig, ax1 = plt_func.subplots(1, numberOfSubplots, constrained_layout=True, figsize=(8*numberOfSubplots, 4))
    _ = ax1.scatter(degrees_lst, MSE_lst)
    _ = ax1.scatter(min_mse, min_val, marker='x', s=40, c=plt_func.rcParams['axes.prop_cycle'].by_key()['color'][0], label='smallest MSE')
    min_val_2std = min_val - 0.2*std_dev
    _ = ax1.axhline(y=min_val_2std, linestyle='dashed', linewidth=0.5, c=plt_func.rcParams['axes.prop_cycle'].by_key()['color'][1], label='0.2 standard deviation')
    _ = ax1.axhline(y=min_val + 0.2*std_dev, linestyle='dashed', linewidth=0.5, c=plt_func.rcParams['axes.prop_cycle'].by_key()['color'][1])
    _ = ax1.set_xlabel(r'degree of polynomial $d$')
    _ = ax1.set_ylabel('Cross-validation MSE')
    _ = ax1.legend()
    
    diff = max(MSE_lst) - min_val_2std
    if diff > 70:
        _ = ax1.set_ylim(ymin = min_val_2std - (min_val_2std % 10) - 10)
    elif diff > 40:
        _ = ax1.set_ylim(ymin = min_val_2std - (min_val_2std % 10))
    else:
        _ = ax1.set_ylim(ymin = min_val_2std - (min_val_2std % 1) - 1)

In [12]:
def fitCubicSplines(sr_X_train, sr_X_test, df_Y_train, df_Y_test, K):
    """
    This function fits a cubic spline with K knots to the training data 
    sr_X_train and df_Y_train. It returns the fitted model and the
    mean squared error due to the test data sr_X_test and df_Y_test.
    
    Args:
        sr_X_train: pandas.core.series.Series
            The Series holding the training data associated with the
            independent variable.
        sr_X_test: pandas.core.series.Series
            The Series holding the test data associated with the
            independent variable.
        df_Y_train: pandas.core.frame.DataFrame
            The DataFrame olding the training data associated with the
            dependent variable.
        df_Y_train: pandas.core.frame.DataFrame
            The DataFrame olding the test data associated with the
            dependent variable.
        K: int
            Number of knots used to fit the cubic spline.
    Returns:
        tuple: (statsmodels.genmod.generalized_linear_model.GLM.fit, MSE)
            The first element of the returned tuple is the fitted model
            on the training data. The second element respresents the test
            MSE.
    """
    
    # https://github.com/pydata/patsy/issues/108
    # https://www.statsmodels.org/dev/generated/statsmodels.gam.smooth_basis.BSplines.html
    # cubic spline with K knots uses has K+4 degrees of freedom
    # so number of basis functions df = K + 3
    df = K + 3
    df_X_transformed_train = dmatrix(f'bs(sr_X_train, df={df}, include_intercept=True)',
                                     {'sr_X_train': sr_X_train}, return_type='dataframe')

    assert df_X_transformed_train.shape[1] == df + 1

    # Build a regular linear model from the splines
    fitted_model = sm.GLM(df_Y_train, df_X_transformed_train).fit()

    sr_Y_pred = fitted_model.predict(dmatrix(f'bs(sr_X_test, df={df}, include_intercept=True)', 
                                       {'sr_X_test': sr_X_test}, return_type='dataframe'))

    MSE = mean_squared_error(df_Y_test, sr_Y_pred)
    
    return fitted_model, MSE

In [13]:
def fitNaturalCubicSplines(sr_X_train, sr_X_test, df_Y_train, df_Y_test, K):
    """
    This function fits a natural cubic spline with K knots to the 
    training data sr_X_train and df_Y_train. It returns the fitted 
    model and the mean squared error due to the test data sr_X_test
    and df_Y_test.
    
    Args:
        sr_X_train: pandas.core.series.Series
            The Series holding the training data associated with the
            independent variable.
        sr_X_test: pandas.core.series.Series
            The Series holding the test data associated with the
            independent variable.
        df_Y_train: pandas.core.frame.DataFrame
            The DataFrame holding the training data associated with the
            dependent variable.
        df_Y_train: pandas.core.frame.DataFrame
            The DataFrame holding the test data associated with the
            dependent variable.
        K: int
            Number of knots used to fit the natural cubic spline.
    Returns:
        tuple: (statsmodels.genmod.generalized_linear_model.GLM.fit, MSE)
            The first element of the returned tuple is the fitted model
            on the training data. The second element respresents the test
            MSE.
    """
    
    # https://github.com/pydata/patsy/issues/108
    # https://www.statsmodels.org/dev/generated/statsmodels.gam.smooth_basis.BSplines.html
    # cubic spline with K knots uses has K+4 degrees of freedom
    # so number of basis functions df = K + 3
    df = K + 3
    df_X_transformed_train = dmatrix(f'cr(sr_X_train, df={df})',
                                     {'sr_X_train': sr_X_train}, return_type='dataframe')

    assert df_X_transformed_train.shape[1] == df + 1

    # Build a regular linear model from the splines
    fitted_model = sm.GLM(df_Y_train, df_X_transformed_train).fit()

    sr_Y_pred = fitted_model.predict(dmatrix(f'cr(sr_X_test, df={df})', 
                                       {'sr_X_test': sr_X_test}, return_type='dataframe'))

    MSE = mean_squared_error(df_Y_test, sr_Y_pred)
    
    return fitted_model, MSE

In [14]:
def plotCubicSpines(df_X, df_Y, dict_models):
    """
    This function plots cubic splines curves with various 
    number of knots all fit on the same data df_X and df_Y.
    
    Args:
        df_X: pandas.core.frame.DataFrame
            The DataFrame holding the dependent variable.
        df_Y: pandas.core.frame.DataFrame
            The DataFrame holding the dependent variable.
        dict_models: dict
            Dictionary that maps the number of knots as 
            keys with values as the fitted model of the type
            statsmodels.genmod.generalized_linear_model.GLM.fit.
    """
    independent = df_X.columns[0]
    dependent = df_Y.columns[0]
    X_pred = np.linspace(df_X[independent].min(), df_X[independent].max(), num=df_X[independent].shape[0], endpoint=True)
    

    fig, ax1 = plt_func.subplots(1, 1, constrained_layout=True, figsize=(8, 4))
    _ = ax1.scatter(df_X[independent], df_Y, s=1)
    for knots, fitted_model in dict_models.items():
        # https://github.com/pydata/patsy/issues/108
        # https://www.statsmodels.org/dev/generated/statsmodels.gam.smooth_basis.BSplines.html
        # cubic spline with K knots uses has K+4 degrees of freedom
        # so number of basis functions df = K + 3
        df = knots + 3
        sr_Y_pred = fitted_model.predict(dmatrix(f'bs(X_pred, df={df}, include_intercept=True)',
                                     {'X_pred': X_pred}, return_type='dataframe'))
        _ = ax1.plot(X_pred, sr_Y_pred, label=f'{knots} knots')
    _ = ax1.legend()
    _ = ax1.set_xlabel(independent)
    _ = ax1.set_ylabel(dependent)

In [15]:
def plotNaturalCubicSpines(df_X, df_Y, dict_models):
    """
    This function plots natural cubic splines curves with 
    various number of knots all fit on the same data df_X 
    and df_Y.
    
    Args:
        df_X: pandas.core.frame.DataFrame
            The DataFrame holding the dependent variable.
        df_Y: pandas.core.frame.DataFrame
            The DataFrame holding the dependent variable.
        dict_models: dict
            Dictionary that maps the number of knots as 
            keys with values as the fitted model of the type
            statsmodels.genmod.generalized_linear_model.GLM.fit.
    """
    independent = df_X.columns[0]
    dependent = df_Y.columns[0]
    X_pred = np.linspace(df_X[independent].min(), df_X[independent].max(), num=df_X[independent].shape[0], endpoint=True)
    

    fig, ax1 = plt_func.subplots(1, 1, constrained_layout=True, figsize=(8, 4))
    _ = ax1.scatter(df_X[independent], df_Y, s=1)
    for knots, fitted_model in dict_models.items():
        # https://github.com/pydata/patsy/issues/108
        # https://www.statsmodels.org/dev/generated/statsmodels.gam.smooth_basis.BSplines.html
        # cubic spline with K knots uses has K+4 degrees of freedom
        # so number of basis functions df = K + 3
        df = knots + 3
        sr_Y_pred = fitted_model.predict(dmatrix(f'cr(X_pred, df={df})',
                                     {'X_pred': X_pred}, return_type='dataframe'))
        _ = ax1.plot(X_pred, sr_Y_pred, label=f'{knots} knots')
    _ = ax1.legend()
    _ = ax1.set_xlabel(independent)
    _ = ax1.set_ylabel(dependent)

In [16]:
def plotWrapperCubicSplines(df_X, df_X_train, df_X_test, df_Y, df_Y_train, df_Y_test, natural):
    """
    This function is a wrapper that plots the test MSE vs 
    the number of knots as a result of fitting (natural) 
    cubic splines.
    
    Args:
        df_X: pandas.core.frame.DataFrame
            The DataFrame holding the dependent variable.
        df_X_train: pandas.core.frame.DataFrame
            The DataFrame holding the training data associated with the
            independent variable.
        df_X_test: pandas.core.frame.DataFrame
            The DataFrame holding the test data associated with the
            independent variable.
        df_Y: pandas.core.frame.DataFrame
            The DataFrame holding the dependent variable.
        df_Y_train: pandas.core.frame.DataFrame
            The DataFrame holding the training data associated with the
            dependent variable.
        df_Y_test: pandas.core.frame.DataFrame
            The DataFrame holding the test data associated with the
            dependent variable.
        natural: bool
            Boolean variable that decide to fit cubic splines (False)
            or natural cubic splines (True).
    """
    independent = df_X.columns[0]
    dict_models = {knots: None for knots in range(1, 10)}
    results = {}
    for knots in dict_models:
        if natural:
            fitted, MSE = fitNaturalCubicSplines(df_X_train[independent], df_X_test[independent], 
                        df_Y_train, df_Y_test, K=knots)
        else:
            fitted, MSE = fitCubicSplines(df_X_train[independent], df_X_test[independent], 
                            df_Y_train, df_Y_test, K=knots)

        dict_models[knots] = fitted
        results[knots] = MSE
        
    knots_lst = list(results.keys())
    MSE_lst = list(results.values())
    std_dev = np.std(MSE_lst)
    min_mse = min(results, key=results.get)
    min_val = results[min_mse]
    knots_lst.remove(min_mse)  # we plot these 'best results' with a different marker
    MSE_lst.remove(min_val)  # we plot these 'best results' with a different marker

    fig, ax1 = plt_func.subplots(1, 1, constrained_layout=True, figsize=(8, 4))
    _ = ax1.scatter(knots_lst, MSE_lst)
    _ = ax1.scatter(min_mse, min_val, marker='x', s=40, c=plt_func.rcParams['axes.prop_cycle'].by_key()['color'][0], label='smallest MSE')
    min_val_2std = min_val - 0.2*std_dev
    _ = ax1.axhline(y=min_val_2std, linestyle='dashed', linewidth=0.5, c=plt_func.rcParams['axes.prop_cycle'].by_key()['color'][1], label='0.2 standard deviation')
    _ = ax1.axhline(y=min_val + 0.2*std_dev, linestyle='dashed', linewidth=0.5, c=plt_func.rcParams['axes.prop_cycle'].by_key()['color'][1])
    _ = ax1.set_xlabel(r'number of knots')
    _ = ax1.set_ylabel('Cross-validation MSE')
    _ = ax1.legend()
    
    diff = max(MSE_lst) - min_val_2std
    if diff > 70:
        _ = ax1.set_ylim(ymin = min_val_2std - (min_val_2std % 10) - 10)
    elif diff > 40:
        _ = ax1.set_ylim(ymin = min_val_2std - (min_val_2std % 10))
    else:
        _ = ax1.set_ylim(ymin = min_val_2std - (min_val_2std % 1) - 1)

    if natural:
        plotNaturalCubicSpines(df_X[[independent]], df_Y, dict_models)
    else:
        plotCubicSpines(df_X[[independent]], df_Y, dict_models)